In [4]:
pip install requests beautifulsoup4

In [79]:
import requests
from bs4 import BeautifulSoup

def search_aliexpress_static(keyword, max_items=10):
    url = f"https://ko.aliexpress.com/w/wholesale-{keyword}.html"
    headers = {
        "User-Agent": "Mozilla/5.0"
    }

    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        print("❌ 요청 실패:", response.status_code)
        return []

    soup = BeautifulSoup(response.text, "html.parser")
    product_blocks = soup.select('div.jr_t')

    results = []
    for block in product_blocks:
        try:
            # 제목
            title_tag = block.find("h3", class_="jr_kp")
            title = title_tag.get_text(strip=True) if title_tag else "제목 없음"

            # 링크 & 이미지
            a_tag = block.find_parent("a", class_="search-card-item")
            if a_tag:
                href = a_tag.get("href", "")
                link = "https:" + href if href.startswith("//") else href

                img_tag = a_tag.find("img", class_="mm_be")
                img_url = img_tag.get("src") if img_tag else ""
                if img_url.startswith("//"):
                    img_url = "https:" + img_url
            else:
                link = "(링크 없음)"
                img_url = ""

            # 가격
            price_spans = block.select("div.jr_kr span")
            price_parts = [span.get_text(strip=True) for span in price_spans]
            price_filtered = [p for p in price_parts if p.isdigit() or p == "."]
            price = "$" + price_filtered[0]+","+price_filtered[1]

            results.append({
                "title": title,
                "price": price,
                "image": img_url,
                "link": link
            })

            if len(results) >= max_items:
                break

        except Exception as e:
            print(f"❌ 상품 블록 파싱 오류: {e}")

    return results

# ✅ 실행 테스트
if __name__ == "__main__":
    keyword = "아이폰16 케이스"
    products = search_aliexpress_static(keyword)
    for i, p in enumerate(products, 1):
        print(f"[{i}] 상품명: {p['title']}")
        print(f"   가격: {p['price']}")
        print(f"   이미지: {p['image']}")
        print(f"   링크: {p['link']}\n")


[1] 상품명: Apple iPhone 15 14 13 12 16 Pro Max 케이스 iPhone 16 Plus 13 12 11 케이스 로고 풀 커버 용 기존 공식 실리콘 케이스
   가격: $2,79
   이미지: https://ae-pic-a1.aliexpress-media.com/kf/S98339bde4dfd404aad6790b346d712dfk.jpg_480x480q75.jpg
   링크: https://ko.aliexpress.com/item/1005008811673198.html?algo_pvid=c98bd008-3a75-4817-948c-0285dcb4809c&algo_exp_id=c98bd008-3a75-4817-948c-0285dcb4809c-0&pdp_ext_f=%7B%22order%22%3A%228221%22%2C%22eval%22%3A%221%22%7D&pdp_npi=4%40dis%21EUR%216.50%212.79%21%21%2151.18%2121.97%21%402140f53817475490061376986eb81b%2112000046766041830%21sea%21NL%210%21ABX&curPageLogUid=ZQlWpbRucp1Z&utparam-url=scene%3Asearch%7Cquery_from%3A

[2] 상품명: 소프트 TPU 실리콘 투명 휴대폰 케이스, 아이폰 16, 11, 12, 13, 14, 15 프로 맥스, XS 맥스, XR, 8, 7 플러스용, 뒷면 커버 투명 케이스
   가격: $1,39
   이미지: https://ae-pic-a1.aliexpress-media.com/kf/Sc2fd635117074c789d949327944db81bn.jpg_480x480q75.jpg
   링크: https://ko.aliexpress.com/item/1005005797570532.html?algo_pvid=c98bd008-3a75-4817-948c-0285dcb4809c&algo_exp_id=c98bd008-3a75-48

In [75]:
# #노력(상세 가격)
# def get_detail_price(url):
#     headers = {
#         "User-Agent": "Mozilla/5.0"
#     }
#     try:
#         res = requests.get(url, headers=headers, timeout=10)

#         # 🔄 HTML 로딩이 느릴 수 있으므로 약간 대기
#         time.sleep(1.5)

#         soup = BeautifulSoup(res.text, "html.parser")

#         # 우선 고정 class 먼저 시도
#         price_tag = soup.find("span", class_=lambda c: c and "product-price-value" in c)
#         if price_tag:
#             return price_tag.get_text(strip=True)

#         # 대체 구조 접근
#         container = soup.select_one("div.price--wrap--Z6Vjgmp")
#         if container:
#             price_span = container.select_one("div.product-price-current span")
#             if price_span:
#                 return price_span.get_text(strip=True)

#         return "❌ 가격 정보 없음"

#     except Exception as e:
#         return f"❌ 오류 발생: {e}"
